In [56]:
import numpy as np
import pandas as pd
import json

In [57]:
df_train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\train.csv',
                       sep=',',
#                       nrows=10000,
                        )
df_train = df_train.loc[:, 'session_id':]
df_train[:3]

,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,b6b4a3c02db0c,1541030408,4JK19KX9RU36,1,search for destination,"Londrina, Brazil",BR,"Londrina, Brazil",desktop,NaN,NaN,NaN
1,b6b4a3c02db0c,1541030410,4JK19KX9RU36,2,search for destination,"Londrina, Brazil",BR,"Londrina, Brazil",desktop,NaN,NaN,NaN
2,5b578bc20be9f,1541030412,KQ7YR1O2APO1,1,search for destination,"Vienna, Austria",RO,"Vienna, Austria",mobile,NaN,NaN,NaN


In [58]:
df_test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\test.csv',
                       sep=',',
#                       nrows=100,
                      )
df_test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,MUOOQC23R8N9,1541453208,9c57f365a02a7,1,change of sort order,interaction sort button,ES,"Lisbon, Portugal",desktop,NaN,NaN,NaN
1,L1IOIJ47C0S6,1541453208,ee9489aaeca70,1,interaction item image,2757973,BR,"Penha, Brazil",desktop,NaN,NaN,NaN
2,L1IOIJ47C0S6,1541453208,ee9489aaeca70,2,interaction item image,2757973,BR,"Penha, Brazil",desktop,NaN,NaN,NaN


In [59]:
import math


GR_COLS = ["user_id", "session_id", "timestamp", "step"]


def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

def add_sum_of_properties(df_pop, df_train):
    merged = df_pop.merge(
        df_train[['reference', 'properties_sum']],
        left_on='reference',
        right_on='reference',
        how='left')
    return merged

def get_popularity(df):
    """Get number of clicks that each item received in the df."""

    mask = df["action_type"] == "clickout item"
    df_clicks = df[mask]
    df_item_clicks = (
        df_clicks
        .groupby("reference")
        .size()
        .reset_index(name="n_clicks")
        .transform(lambda x: x.astype(int))
    )

    return df_item_clicks


def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out


def explode(df_in, col_expl):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_expl] = df[col_expl].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    if col_expl == 'impressions':
        df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns.drop(col_expl)}
    )
    
#     df_out_prices = pd.DataFrame(
#         {col: np.repeat(df[col].values,
#                         df['prices'].str.len())
#          for col in df.columns.drop('prices')}
#     )
    
#     df_out['prices'] = df_out_prices['prices'].values

    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, col_expl] = df_out[col_expl].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out


def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out


def calc_recommendation(df_expl, df_pop):
    """Calculate recommendations based on popularity of items.

    The final data frame will have an impression list sorted according to the number of clicks per item in a reference data frame.

    :param df_expl: Data frame with exploded impression list
    :param df_pop: Data frame with items and number of clicks
    :return: Data frame with sorted impression list according to popularity in df_pop
    """

    # merg, przypisanie liczby klików w item na podstawie df z popularity
    df_expl_clicks = (
        df_expl[GR_COLS + ["impressions"]]
        .merge(df_pop,
               left_on="impressions",
               right_on="reference",
               how="left")
    )

    # zmiana impressions z typu float64 na str (object)
    df_out = (
        df_expl_clicks
        .assign(impressions=lambda x: x["impressions"].apply(str))
        .sort_values(GR_COLS + ["n_clicks"],
                     ascending=[True, True, True, True, False])
    )

    df_out = group_concat(df_out, GR_COLS, "impressions")
    df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)

    return df_out


In [60]:
df_popular = get_popularity(df_train)

In [61]:
# df_popular_and_sum_properties = add_sum_of_properties(df_popular, df_train)

In [62]:
df_target = get_submission_target(df_test)

In [63]:
df_expl = explode(df_target, "impressions")

In [64]:
df_expl[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016


In [65]:
def calc_recommendation2(df_expl, df_pop):
    """Calculate recommendations based on popularity of items.

    The final data frame will have an impression list sorted according to the number of clicks per item in a reference data frame.

    :param df_expl: Data frame with exploded impression list
    :param df_pop: Data frame with items and number of clicks
    :return: Data frame with sorted impression list according to popularity in df_pop
    """
    # merg, przypisanie liczby klików w item na podstawie df z popularity
    df_expl_clicks = (
        df_expl[GR_COLS + ["impressions", 'prices', 'number_of_impressions'] ]
        .merge(df_pop,
               left_on="impressions",
               right_on="reference",
               how="left")
        .drop_duplicates()
    )
    df_with_max_clicks_in_impressions = pd.DataFrame(df_expl_clicks.groupby(['session_id'])['n_clicks'].max())
    df_expl_clicks = df_expl_clicks.merge(df_with_max_clicks_in_impressions, left_on='session_id', right_on='session_id', how='left')
    df_expl_clicks = df_expl_clicks.rename(columns={'n_clicks_y':"max_clicks_in_impression", 'n_clicks_x':"n_clicks"})

    # zmiana impressions z typu float64 na str (object)
    df_out = (
        df_expl_clicks
        .assign(impressions=lambda x: x["impressions"].apply(str))
        .sort_values(GR_COLS + ["prices"],
                     ascending=[True, True, True, True, True])
    )
    

    df_out['value_n_clicks'] = (df_out['n_clicks'] - df_out['n_clicks'].mean())/df_out['n_clicks'].std()
    df_out['value_properties'] = (df_out['properties_sum'] - df_out['properties_sum'].mean())/df_out['properties_sum'].std()
    df_out['value_prices'] = -(df_out['prices'] - df_out['prices'].mean())/df_out['prices'].std()

    df_out['sum_all'] = (df_out['value_n_clicks'] + df_out['value_properties'] + df_out['value_prices'])
    
    df_out = df_out.sort_values(GR_COLS + ["sum_all"], ascending=[True, True, True, True, False])
#     df_out = group_concat(df_out, GR_COLS, "impressions")
#     df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)

    return df_out

In [ ]:
out = calc_recommendation2(df_expl, df_popular_and_sum_properties)

In [ ]:
out[:30]

In [ ]:
#2
out[60:100]

In [31]:
# mean price

In [66]:
def mean_price(df):
    return df['prices'].mean()

In [67]:
df_with_mean_price = df_expl.groupby(['session_id', 'timestamp']).apply(mean_price).to_frame("mean_price")
df_with_mean_price[:3]

,,mean_price
session_id,timestamp,
000056cd97ce2,1541484101,58.0400
00011005327e1,1541539943,77.8125
0001fe0007954,1541511705,90.7600


In [68]:
df_merged_with_mean_price = df_expl.merge(df_with_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
df_merged_with_mean_price[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,mean_price
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,58.04
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,58.04
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,58.04


In [69]:
df_merged_with_mean_price['price_mean_difference'] = abs(df_merged_with_mean_price['prices'] - df_merged_with_mean_price['mean_price'])

In [70]:
df_merged_with_mean_price[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,mean_price,price_mean_difference
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,58.04,128.96
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,58.04,19.96
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,58.04,41.96


In [84]:
df_merged_with_mean_price = df_merged_with_mean_price.sort_values(['session_id', 'timestamp', 'price_mean_difference'], ascending=[True, True, True])
df_merged_with_mean_price[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,mean_price,price_mean_difference
4,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,150,25,1339848,58.04,91.96000000000001
3,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,51,25,1153426,58.04,7.039999999999999
13,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,52,25,1837709,58.04,6.039999999999999


In [85]:
df_merged_with_mean_price = df_merged_with_mean_price.astype({'mean_price':str, 'price_mean_difference':str,'impressions':str})

In [86]:
df_out = group_concat(df_merged_with_mean_price, ['user_id','session_id', 'timestamp', 'step'], "impressions")
df_out = df_out.rename(columns={'impressions':'item_recommendations'})
df_out[:3]

,user_id,session_id,timestamp,step,item_recommendations
0,000I430EXZC0,ddabdf53a4d38,1541501642,1,103495 103658 980287 103600 824196 98265 10358...
1,000O8CPC6T0W,846b712ae8e1f,1541480658,1,6221916 6506 6507 5961668 1160523 1151391 1492...
2,000REY2YYH1D,4a313f2eb8954,1541465923,1,63809 57541 60452 70266 355861 65392 56901 886...


In [87]:
df_out.to_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_popular.csv', sep=',', index=False)
